In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, os, django
from geotext import GeoText

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
from sklearn.manifold import TSNE
from scoping.models import *
from scoping.views import *
from tmv_app.views import *

now=datetime.datetime.now()

In [3]:
q = Query.objects.get(pk=7966)
p = q.project
tag = Tag.objects.get(pk=5150)

In [4]:
dos = DocOwnership.objects.filter(user__username="minj",query=q,tag=tag)
docs = dos.values_list('doc')
dos.count()

85

In [5]:
cats = Category.objects.filter(project=p).exclude(name__icontains="hidden>")
df = pd.DataFrame.from_dict(
    DocUserCat.objects.filter(
        category__in=cats,
        doc__in=docs
    ).values(
        'doc__id',
        'category__name',
        'category__level',
        'user__username'
    )
)
df['category__level'] = df['category__level'].astype(str)
df['cat'] = df[['category__level','category__name','user__username']].apply(lambda x: ' - '.join(x), axis=1)
df['val'] = 1
df.head()

,doc__id,category__name,category__level,user__username,cat,val
0,738460,Mitigation,1,maowenxin2014@163.com,1 - Mitigation - maowenxin2014@163.com,1
1,738460,Governance & policies,2,maowenxin2014@163.com,2 - Governance & policies - maowenxin2014@163.com,1
2,738460,Economic instruments,3,maowenxin2014@163.com,3 - Economic instruments - maowenxin2014@163.com,1
3,738460,Emissions trading,4,maowenxin2014@163.com,4 - Emissions trading - maowenxin2014@163.com,1
4,738460,National,5,maowenxin2014@163.com,5 - National - maowenxin2014@163.com,1


In [6]:
wide_df = (df[['doc__id','cat','val']]
           .drop_duplicates()
           .pivot(index='doc__id',columns="cat",values='val')
           .fillna(0)
          )
cs = wide_df.columns
wide_df['doc_id'] = wide_df.index
wide_df = wide_df.reset_index(drop=True)
wide_df.head()


cat,1 - Adaptation - maowenxin2014@163.com,1 - Adaptation - minj,1 - Mitigation - maowenxin2014@163.com,1 - Mitigation - minj,2 - Governance & policies - maowenxin2014@163.com,2 - Governance & policies - minj,3 - Economic instruments - maowenxin2014@163.com,3 - Economic instruments - minj,3 - Goal setting - minj,3 - Information policies - maowenxin2014@163.com,...,4 - Reports and assessments/ monitoring and evaluation - maowenxin2014@163.com,4 - Subsidy/compensation/ allowance - maowenxin2014@163.com,4 - Tax - maowenxin2014@163.com,4 - Tax - minj,5 - Company & organizations - maowenxin2014@163.com,5 - Local - maowenxin2014@163.com,5 - National - maowenxin2014@163.com,5 - Sub-national/ regional - maowenxin2014@163.com,5 - Supranational & international - maowenxin2014@163.com,doc_id
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2991
1,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,32357
2,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40091
3,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57458
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,60695


In [7]:
double_dos = DocOwnership.objects.filter(
    doc__in=docs,query=q,user__username="minj"
).exclude(tag=tag).values_list('pk',flat=True)
do_df = pd.DataFrame.from_dict(
    DocOwnership.objects.filter(doc__in=docs, query=q,relevant__gt=0).exclude(pk__in=double_dos).values(
        'doc__id',
        'user__username',
        'relevant'
    )
)
do_df = (
    do_df.drop_duplicates(['doc__id','user__username'])
    .pivot(index='doc__id',columns="user__username",values="relevant")
)
rs = []
for c in do_df.columns:
    do_df = do_df.rename(columns={c: f'0 - relevant - {c}'})
    rs.append(f'0 - relevant - {c}')
do_df['doc_id'] = do_df.index
do_df = do_df.reset_index(drop=True)

do_df.head()

user__username,0 - relevant - maowenxin2014@163.com,0 - relevant - minj,doc_id
0,1.0,3.0,2991
1,1.0,1.0,16076
2,1.0,1.0,32357
3,1.0,1.0,40091
4,1.0,1.0,57458


In [8]:
doc_df = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=docs).values(
        'id','title','content'
    )
).rename(columns={"id":"doc_id"})
doc_df.head()

,doc_id,title,content
0,738460,Offsetting benefits? Analyzing access to fores...,Emissions trading has created new forms of exc...
1,40091,An overview of the science-policy interface am...,Global progress in addressing climate change t...
2,1291618,Effect of carbon tax on the industrial competi...,As one of the most cost-effective means of car...
3,187387,The liability rules under international GHG em...,Article 17 or the Kyoto Protocol authorizes em...
4,219972,Avoiding bio-perversity from carbon sequestrat...,The development of a new carbon economy has th...


In [9]:
df = pd.merge(doc_df,pd.merge(wide_df,do_df))[["doc_id","title","content"]+list(rs)+list(cs)]
df.head()

,doc_id,title,content,0 - relevant - maowenxin2014@163.com,0 - relevant - minj,1 - Adaptation - maowenxin2014@163.com,1 - Adaptation - minj,1 - Mitigation - maowenxin2014@163.com,1 - Mitigation - minj,2 - Governance & policies - maowenxin2014@163.com,...,"4 - Rationing, bans & moratoria - maowenxin2014@163.com",4 - Reports and assessments/ monitoring and evaluation - maowenxin2014@163.com,4 - Subsidy/compensation/ allowance - maowenxin2014@163.com,4 - Tax - maowenxin2014@163.com,4 - Tax - minj,5 - Company & organizations - maowenxin2014@163.com,5 - Local - maowenxin2014@163.com,5 - National - maowenxin2014@163.com,5 - Sub-national/ regional - maowenxin2014@163.com,5 - Supranational & international - maowenxin2014@163.com
0,738460,Offsetting benefits? Analyzing access to fores...,Emissions trading has created new forms of exc...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,40091,An overview of the science-policy interface am...,Global progress in addressing climate change t...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1291618,Effect of carbon tax on the industrial competi...,As one of the most cost-effective means of car...,NaN,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,187387,The liability rules under international GHG em...,Article 17 or the Kyoto Protocol authorizes em...,NaN,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,219972,Avoiding bio-perversity from carbon sequestrat...,The development of a new carbon economy has th...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df[df['doc_id']==423432]

,doc_id,title,content,0 - relevant - maowenxin2014@163.com,0 - relevant - minj,1 - Adaptation - maowenxin2014@163.com,1 - Adaptation - minj,1 - Mitigation - maowenxin2014@163.com,1 - Mitigation - minj,2 - Governance & policies - maowenxin2014@163.com,...,"4 - Rationing, bans & moratoria - maowenxin2014@163.com",4 - Reports and assessments/ monitoring and evaluation - maowenxin2014@163.com,4 - Subsidy/compensation/ allowance - maowenxin2014@163.com,4 - Tax - maowenxin2014@163.com,4 - Tax - minj,5 - Company & organizations - maowenxin2014@163.com,5 - Local - maowenxin2014@163.com,5 - National - maowenxin2014@163.com,5 - Sub-national/ regional - maowenxin2014@163.com,5 - Supranational & international - maowenxin2014@163.com
16,423432,Cost-effectiveness of high-efficiency applianc...,This paper presents an analysis of the cost-ef...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df.to_csv('doctables/comparison.csv', index=False)

In [17]:
cs = ["doc_id"] + [x for x in df.columns if "wenxin" in x]
df[pd.isna(df['0 - relevant - maowenxin2014@163.com'])][cs]


,doc_id,0 - relevant - maowenxin2014@163.com,1 - Adaptation - maowenxin2014@163.com,1 - Mitigation - maowenxin2014@163.com,2 - Governance & policies - maowenxin2014@163.com,3 - Economic instruments - maowenxin2014@163.com,3 - Information policies - maowenxin2014@163.com,3 - International policies - maowenxin2014@163.com,3 - Planning and public provisioning/services - maowenxin2014@163.com,3 - Regulatory instruments - maowenxin2014@163.com,...,4 - Public information/ awareness campaigns - maowenxin2014@163.com,"4 - Rationing, bans & moratoria - maowenxin2014@163.com",4 - Reports and assessments/ monitoring and evaluation - maowenxin2014@163.com,4 - Subsidy/compensation/ allowance - maowenxin2014@163.com,4 - Tax - maowenxin2014@163.com,5 - Company & organizations - maowenxin2014@163.com,5 - Local - maowenxin2014@163.com,5 - National - maowenxin2014@163.com,5 - Sub-national/ regional - maowenxin2014@163.com,5 - Supranational & international - maowenxin2014@163.com
2,1291618,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,187387,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,254719,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1808191,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,184733,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,446240,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,225781,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,607725,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,762844,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,356639,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
Doc.objects.get(pk=423432).title

'Cost-effectiveness of high-efficiency appliances in the U.S. residential sector: A case study'

In [18]:
df[df['0 - relevant - maowenxin2014@163.com']==0][cs]

,doc_id,0 - relevant - maowenxin2014@163.com,1 - Adaptation - maowenxin2014@163.com,1 - Mitigation - maowenxin2014@163.com,2 - Governance & policies - maowenxin2014@163.com,3 - Economic instruments - maowenxin2014@163.com,3 - Information policies - maowenxin2014@163.com,3 - International policies - maowenxin2014@163.com,3 - Planning and public provisioning/services - maowenxin2014@163.com,3 - Regulatory instruments - maowenxin2014@163.com,...,4 - Public information/ awareness campaigns - maowenxin2014@163.com,"4 - Rationing, bans & moratoria - maowenxin2014@163.com",4 - Reports and assessments/ monitoring and evaluation - maowenxin2014@163.com,4 - Subsidy/compensation/ allowance - maowenxin2014@163.com,4 - Tax - maowenxin2014@163.com,5 - Company & organizations - maowenxin2014@163.com,5 - Local - maowenxin2014@163.com,5 - National - maowenxin2014@163.com,5 - Sub-national/ regional - maowenxin2014@163.com,5 - Supranational & international - maowenxin2014@163.com
